## Three eDSLs for business logic
#####  "I wanted to do more things with my language, so I had no choice but to make it smaller" - Connor McBride (@pigworker)
#### Lef Ioannidis (@elefthei) : Investment Engineer, Bridgewater Associates

### First some minimal dependencies.

In [1]:
%classpath add mvn joda-time joda-time 2.10.5
%classpath add mvn org.typelevel cats-core_2.12 2.1.1
// Some imports first
import org.joda.time.DateTime
import org.joda.time.Duration
import org.joda.time.format.DateTimeFormat
import java.io.File
import org.joda.time.format.DateTimeFormatter
import scala.io.Source
import scala.collection.mutable.ArrayBuffer
import scala.math.Ordered._

// Dates are ordered
implicit val DateTimeOrdered = new Ordering[DateTime] {
  override def compare(x: DateTime, y: DateTime): Int = x.getMillis.compare(y.getMillis)
}

$line25.$read$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$iw$$anon$1@66686185

### We will be working with Period series.

#### PeriodSeries are Arrays of Doubles. Each number corresponds to a constant-frequency interval and they start at some base date. 

### Let's define a Frequency for our PeriodSeries

In [ ]:
object Frequency extends Enumeration {
  type Inner = Value
  // Can be one of the following
  val Daily, Monthly, Quarterly, Yearly = Value
  def fromString(s: String): Frequency =
    s match {
      case "Yearly" => Yearly
      case "Monthly" => Monthly
      case "Quarterly" => Quarterly
      case "Daily" => Daily
    }
}
// Alias for Frequency type
type Frequency = Frequency.Inner
// Days is the smallest unit
implicit class FreqToDays(f: Frequency) {
  def days(): Int =
    f match {
      case Frequency.Monthly => 30
      case Frequency.Quarterly => 30 * 3
      case Frequency.Yearly => 365
      case _ => 1
    }
}
// Frequencies are ordered
implicit val FrequencyOrdered = new Ordering[Frequency]{
  override def compare(a: Frequency, b: Frequency): Int = a.days.compare(b.days)
}

### Let's start with a naive definition for PeriodSeries

In [ ]:
case class PeriodSeries(data: Array[Double], frequency: Frequency, base: DateTime)

In [ ]:
// Load CSV data
object CSV {
  def load(csv: String): PeriodSeries = {
    val file = Source.fromFile(csv).getLines().toSeq
    val freq = Frequency.fromString(file.head)
    val base = DateTime.parse(file.tail.head, DateTimeFormat.forPattern("yyyy"))
    val data = file.tail.tail.map(_.toDouble).toArray
    new PeriodSeries(data, freq, base)
  }
}

// Plot them
implicit class PeriodSeriesPlotting(ps: PeriodSeries) {
  def toTicks: Seq[(DateTime, Double)] = 
    ps.data.toIndexedSeq.zipWithIndex.map { 
      case (d, i) => (ps.base.plusDays(ps.frequency.days * i), d) 
    }

  def plot(name: String, yname: String): SimpleTimePlot = 
    new SimpleTimePlot {
      title = name
      data = toTicks.map { case(t, d) => Map(yname -> d, "time" -> t.toDate()) }
      columns = Seq(yname)
    }
  def plot(): SimpleTimePlot = plot("Timeseries", "y")
}

### PeriodSeries can represent a wide range of financial data.
#### For example: Prices of ACME stocks over a period of time.

In [ ]:
val stocks: PeriodSeries = CSV.load("acme-stocks/stocks-2.txt")
stocks.plot("Acme Stocks (2018)", "Prices")

### Cool! Of course we want to compute some elementary statistics about our PeriodSeries

In [ ]:
implicit class PeriodSeriesMath(ps: PeriodSeries) {
  import scala.math.Ordered._
  def last: Double = ps.data.last
  def first: Double = ps.data.head
  def enddate: DateTime =
    ps.base.plusDays(ps.frequency.days() * ps.data.length)
  def vol: Double = {
    val avg = ps.data.sum / ps.data.length
    math.sqrt(ps.data.map(a => math.pow(a - avg, 2)).sum / ps.data.length)
  }
  def roll(f: PeriodSeries => Double, window: Int = 2): PeriodSeries = {
    require(ps.data.length > window, "Window too big for small dataset")
    var i = 0
    val result: ArrayBuffer[Double] = ArrayBuffer()
    while (i + window < ps.data.length) {
      val newps = ps.copy(data = ps.data.slice(i, i + window))
      result.append(f(newps))
      i += 1
    }
    ps.copy(data = result.toArray)
  }
  def rollVol(window: Int = 2): PeriodSeries = roll(_.vol, window)
  def intersect(other: PeriodSeries, f: (Double, Double) => Double): PeriodSeries = {
    val ord = Ordering[DateTime]
    require(ps.frequency == other.frequency, "Cannot intersect PeriodSeries of different frequencies")
    require(ord.max(ps.base, other.base) <= ord.min(ps.enddate, other.enddate), "Cannot add PeriodSeries that do not overlap")
    ps.copy(data = ps.data.zip(other.data).map(p => f(p._1, p._2)), base = ord.max(ps.base, other.base))
  }
  def +(other: PeriodSeries): PeriodSeries = intersect(other, _ + _)
  def -(other: PeriodSeries): PeriodSeries = intersect(other, _ - _)
  def *(other: PeriodSeries): PeriodSeries = intersect(other, _ * _)
  def /(other: PeriodSeries): PeriodSeries = intersect(other, _ / _)
}

In [ ]:
stocks.vol

In [ ]:
stocks.rollVol(12).plot()

### Let's give your PeriodSeries some more powers
#### Upsampling, Downsampling

In [ ]:
  implicit class PeriodSeriesExtensions(ps: PeriodSeries) {
    import scala.math.Ordered._
    def downSample(newfreq: Frequency) : PeriodSeries = {
      require(newfreq >= ps.frequency, "Cannot down-sample with higher frequency")
      val step = (newfreq.days / ps.frequency.days).toInt
      def groupByNum[A](s: Iterable[A], n: Int): Iterable[Iterable[A]] =
        if(n == 0) {
          Seq()
        } else if(s.size <= n) {
          Seq(s)
        } else {
          val (left, right) = s.splitAt(n)
          Seq(left) ++ groupByNum(right.tail, n)
        }
      ps.copy(data = groupByNum(ps.data, step).map(data => data.sum / data.size).toArray, frequency = newfreq)
    }

    private def interpolate(first: Double, second: Double, steps: Int): Array[Double] =
      (0 to steps).map(_ * (second - first) / steps).map(_ + first).toArray

    def upSample(newfreq: Frequency): PeriodSeries = {
      require(newfreq <= ps.frequency, "Cannot up-sample with lower frequency")
      val steps = ps.frequency.days / newfreq.days
      val seq = ps.data.toIndexedSeq.zipWithIndex.flatMap {
        case (d, i) if i < ps.data.length - 1 => interpolate(d, ps.data(i + 1), steps)
        case (d, _) => Array(d)
      }.toArray
      ps.copy(data = seq, frequency = newfreq)
    }
  }

### So far so good! Now let's experiment with our new Period Series implementation.
#### Things to consider
1. Are the results correct?
2. What are the business logic pitfalls and how can we prevent them?
3. We would like full **Provenance**, is it even possible?

### How about the following two transformations?
#### Did you expect them to be different? What happened?

In [ ]:
stocks.upSample(Frequency.Daily).rollVol(12).plot()

In [ ]:
stocks.rollVol(12).upSample(Frequency.Daily).plot()

### Can we use types so the wrong business logic does not typecheck?
### How about all these runtime errors?

In [ ]:
val stocks = CSV.load("acme-stocks/stocks-1.txt")
val other = CSV.load("acme-stocks/stocks-1-quarterly.txt")

In [ ]:
stocks / other

In [ ]:
stocks - other

### Runtime errors
1. Waste time and concentration
2. Can trigger at the worst times and are hard to document.
3. Typed error approaches like Try[T] and ZIO are great, but still have to think about runtime errors all the time.
4. **Compile-time errors >> Runtime errors** when possible
5. Sometimes it's a trade-off complexity/correctness